In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open("../data/names.txt", 'r').read().splitlines()
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {s: i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}

In [4]:
block_sz = 3
X, Y = [], []

for w in words[:5]:

    print("---")
    print(w)
    print("---")
    context = [0] * block_sz
    for ch in w + ".":
        ix = stoi[ch]
        print(''.join(itos[i] for i in context), " ---> ", itos[ix])
        context = context[1:] + [ix]

---
emma
---
...  --->  e
..e  --->  m
.em  --->  m
emm  --->  a
mma  --->  .
---
olivia
---
...  --->  o
..o  --->  l
.ol  --->  i
oli  --->  v
liv  --->  i
ivi  --->  a
via  --->  .
---
ava
---
...  --->  a
..a  --->  v
.av  --->  a
ava  --->  .
---
isabella
---
...  --->  i
..i  --->  s
.is  --->  a
isa  --->  b
sab  --->  e
abe  --->  l
bel  --->  l
ell  --->  a
lla  --->  .
---
sophia
---
...  --->  s
..s  --->  o
.so  --->  p
sop  --->  h
oph  --->  i
phi  --->  a
hia  --->  .


In [5]:
def create_dataset(words, block_sz=3):
    X, Y = [], []
    for w in words:
        context = [0] * block_sz
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

In [6]:
len(words)

32033

In [7]:
import random

random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = create_dataset(words[:n1])
Xdev, Ydev = create_dataset(words[n1:n2])
Xte, Yte = create_dataset(words[n2:])

In [8]:
Xtr.shape, Xdev.shape, Xte.shape

(torch.Size([182775, 3]), torch.Size([22718, 3]), torch.Size([22653, 3]))

In [9]:
g = torch.Generator().manual_seed(214783647)
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [10]:
sum(p.nelement() for p in parameters)

11897

In [33]:
for p in parameters:
    p.requires_grad = True

#### Loss when the predicted proabilities are uniform

In [31]:
with torch.no_grad():
    probs = torch.tensor([1/27] * 27)
    loss = F.cross_entropy(probs, Ytr[5])
    print(loss.item())

3.295836925506592


### Training parameters to give out uniform probabilites

In [132]:
g = torch.Generator().manual_seed(214783647)
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [133]:
for p in parameters:
    p.requires_grad = True

In [134]:
target = torch.ones((27, )) / 27

In [135]:
target

tensor([0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370,
        0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370,
        0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370])

In [136]:
target = target.repeat((32, 1)); target.shape

torch.Size([32, 27])

Here, we are training the parameters to output uniform probabilities.

In [137]:
for _ in range(7000):
    ix = torch.randint(0, Xtr.shape[0], (32, ))
    
    # forward pass
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.mse_loss(logits, target)

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    # print(loss.item())

    lr = 0.1
    for p in parameters:
        p.data -= lr * p.grad

In [138]:
with torch.no_grad():
    emb = C[Xtr]
    h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr)
    print(loss.item())

3.326951503753662


In [139]:
logits[0]

tensor([-0.0886,  0.0328,  0.1360,  0.0213,  0.0158,  0.0285,  0.0105,  0.0119,
         0.0075,  0.0208,  0.0882, -0.0239, -0.0518,  0.1522, -0.0452,  0.0267,
         0.0414,  0.0677,  0.0421,  0.0052,  0.1898,  0.0553,  0.1096,  0.0186,
         0.0847,  0.1009,  0.0044])

In [140]:
with torch.no_grad():
    loss = F.cross_entropy(logits[1234], Ytr[5])
    print(loss.item())

3.3410379886627197


In [141]:
lossi = []
stepi = [0]

In [162]:
for i in range(25000):

    # minibatch
    ix = torch.randint(0, Xtr.shape[0], (32, ))
    
    # forward pass
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    lr = 0.01
    for p in parameters:
        p.data -= lr * p.grad

    stepi.append(stepi[-1]+1)
    lossi.append(loss.log10().item())

# print(loss.item())

In [163]:
plt.plot(stepi[1:], lossi)

In [164]:
with torch.no_grad():
    emb = C[Xtr]
    h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr)
    print(loss.item())

2.129706621170044


In [165]:
with torch.no_grad():
    emb = C[Xdev]
    h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ydev)
    print(loss.item())

2.179611921310425


In [204]:
plt.figure(figsize=(8, 8))
plt.scatter(C[:, 0].data, C[:, 1].data, s=200)
for i in range(C.shape[0]):
    plt.text(C[i, 0].item(), C[i, 1].item(), itos[i], ha="center", va="center", color="white")
plt.grid("minor")